In [2]:
import pandas as pd
import numpy as np
import torch
import os
import pickle as pickle
from transformers import (
    XLMRobertaForSequenceClassification,
    XLMRobertaConfig,
    XLMRobertaTokenizer,
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    RobertaConfig,
    RobertaTokenizer,
    RobertaForSequenceClassification,
    BertTokenizer,
    EarlyStoppingCallback,
)
from load_data import *
from utils import *
import random

In [3]:
def seed_everything(seed: int = 42):
    """
    Fix all related seeds

    :param seed: 42 (default)
    :return:
    """
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything()

In [4]:
def label_to_num(label):
    num_label = []
    with open("dict_label_to_num.pkl", "rb") as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])
    return num_label

In [42]:
raw_df = pd.read_csv("/opt/ml/code/train copy.csv")
SAVEDIR = './EDA_analysis_result'

# Raw data 구성

In [6]:
raw_df

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [7]:
raw_df.label.describe()

count           32470
unique             30
top       no_relation
freq             9534
Name: label, dtype: object

In [8]:
from collections import Counter
labels_int= []
nums= []
for k_, v_ in sorted([(k, v) for k,v in Counter(label_to_num(raw_df["label"].values)).items()], key=lambda x:x[0]):
    labels_int.append(k_)
    nums.append(v_)

In [9]:
label_num_df = pd.DataFrame({"label":labels_int, "count":nums})
label_num_df

,label,count
0,0,9534
1,1,4284
2,2,420
3,3,380
4,4,2103
5,5,1320
6,6,3573
7,7,1195
8,8,139
9,9,48


In [10]:
label_27_df = raw_df[raw_df.label.eq("per:place_of_death")]
len(label_27_df)

40

In [11]:
label_26_df = raw_df[raw_df.label.eq("per:place_of_birth")]
label_26_df

,id,sentence,subject_entity,object_entity,label,source
91,91,"백한성(白漢成, 水原鶴人, 1899년 6월 15일 조선 충청도 공주 출생 ~ 197...","{'word': '백한성', 'start_idx': 0, 'end_idx': 2, ...","{'word': '조선 충청도 공주', 'start_idx': 28, 'end_id...",per:place_of_birth,wikipedia
774,774,코치시 졸탄(1952년 5월 30일 ~ 2016년 11월 6일)은 헝가리 태생의 피...,"{'word': '코치시 졸탄', 'start_idx': 0, 'end_idx': ...","{'word': '헝가리', 'start_idx': 37, 'end_idx': 39...",per:place_of_birth,wikipedia
1105,1105,"윤호중은 경기도 장단군 출신으로, 배재고등보통학교에서 미술 교사였던 조각가 김복진에...","{'word': '윤호중', 'start_idx': 0, 'end_idx': 2, ...","{'word': '경기도', 'start_idx': 5, 'end_idx': 7, ...",per:place_of_birth,wikipedia
1110,1110,토미 메이컴은 북아일랜드 카운티 아마의 키디에서 1932년에 태어났다.,"{'word': '토미 메이컴', 'start_idx': 0, 'end_idx': ...","{'word': '북아일랜드 카운티 아마', 'start_idx': 8, 'end_...",per:place_of_birth,wikipedia
1300,1300,빌리 오르반은 독일 카이저슬라우테른에서 헝가리인 아버지와 폴란드인 어머니 사이에서 ...,"{'word': '빌리 오르반', 'start_idx': 0, 'end_idx': ...","{'word': '카이저슬라우테른', 'start_idx': 11, 'end_idx...",per:place_of_birth,wikipedia
...,...,...,...,...,...,...
31936,31936,본관은 성주(星州)이고 호(號)는 산남(山南)이며 영어 이름은 토머스 도(Thoma...,"{'word': '토머스 도', 'start_idx': 35, 'end_idx': ...","{'word': '성주(星州)', 'start_idx': 4, 'end_idx': ...",per:place_of_birth,wikipedia
32025,32025,루자인 알하틀룰은 2015년 12월에 사우디아라비아 역사상 최초로 여성이 선거권을 ...,"{'word': '루자인 알하틀룰', 'start_idx': 0, 'end_idx'...","{'word': '사우디아라비아', 'start_idx': 21, 'end_idx'...",per:place_of_birth,wikipedia
32162,32162,1946년 평안북도 진남포 (현재의 남포특별시)라고 알려져 있지만 공식적으로 평안남...,"{'word': '하용조', 'start_idx': 67, 'end_idx': 69...","{'word': '평안남도 강서군', 'start_idx': 43, 'end_idx...",per:place_of_birth,wikipedia
32204,32204,"툰쿠 압둘 라만은 1903년 2월 8일, 말레이시아의 알로르스타르에서 태어났다.","{'word': '툰쿠 압둘 라만', 'start_idx': 0, 'end_idx'...","{'word': '말레이시아', 'start_idx': 23, 'end_idx': ...",per:place_of_birth,wikipedia


In [12]:
label_11_df = raw_df[raw_df.label.eq("per:place_of_residence")]
label_11_df

,id,sentence,subject_entity,object_entity,label,source
167,167,이어 “6월 항쟁의 미완성을 통해 민주주의 후퇴를 겪었다. 국민들이 촛불혁명으로 탄...,"{'word': '문재인', 'start_idx': 50, 'end_idx': 52...","{'word': '대한민국', 'start_idx': 83, 'end_idx': 8...",per:place_of_residence,wikitree
230,230,다만 역법의 시작을 이슬람력과 동일하게 무함마드 및 그의 추종자들이 메카에서 메디나...,"{'word': '무함마드', 'start_idx': 22, 'end_idx': 2...","{'word': '메디나', 'start_idx': 43, 'end_idx': 45...",per:place_of_residence,wikipedia
233,233,"유한굉(劉漢宏, Liu Hanhong, ~ 887년)은 중국 당나라 말기에 활약했던...","{'word': '유한굉', 'start_idx': 0, 'end_idx': 2, ...","{'word': '중국 당나라', 'start_idx': 31, 'end_idx':...",per:place_of_residence,wikipedia
315,315,전쟁은 일본이 개항하면서 에도 막부(도쿠가와 막부)의 이이 나오스케(히코네 번)가 ...,"{'word': '이이 나오스케', 'start_idx': 30, 'end_idx'...","{'word': '히코네 번', 'start_idx': 38, 'end_idx': ...",per:place_of_residence,wikipedia
503,503,김성주(1990년 11월 15일 ~)은 대한민국의 축구 선수이며 현재 K리그1 인천...,"{'word': '김성주', 'start_idx': 0, 'end_idx': 2, ...","{'word': '대한민국', 'start_idx': 22, 'end_idx': 2...",per:place_of_residence,wikipedia
...,...,...,...,...,...,...
31914,31914,"대구광역시에서는 전 지역에서 홍준표가 앞섰는데, 유승민 후보의 거점 지역 동구, 한...","{'word': '김부겸', 'start_idx': 73, 'end_idx': 75...","{'word': '대구광역시', 'start_idx': 0, 'end_idx': 4...",per:place_of_residence,wikipedia
31975,31975,"유력자의 협력 하에 필리핀 헌법을 만들고, 호세 라우렐을 대통령으로 내세워 일본에 ...","{'word': '호세 라우렐', 'start_idx': 24, 'end_idx':...","{'word': '필리핀', 'start_idx': 11, 'end_idx': 13...",per:place_of_residence,wikipedia
32127,32127,"또 더불어민주당 조승래(대전 유성갑) 의원이 12건(12.3%), 더불어민주당 김해...","{'word': '김해영', 'start_idx': 44, 'end_idx': 46...","{'word': '부산 연제구', 'start_idx': 48, 'end_idx':...",per:place_of_residence,wikitree
32131,32131,"한편, 김헌창(金憲昌)이 822년 3월에 반란을 일으켜, 웅진(熊津)을 수도로 하고...","{'word': '김헌창', 'start_idx': 4, 'end_idx': 6, ...","{'word': '웅진(熊津)', 'start_idx': 32, 'end_idx':...",per:place_of_residence,wikipedia


In [13]:
sub_entity_words = [entity['word'] for entity in map(eval,raw_df.subject_entity.values)]
obj_entity_words = [entity['word'] for entity in map(eval,raw_df.object_entity.values)]

In [14]:
test_df = pd.DataFrame({ "sub_entity_words" :sub_entity_words, 'obj_entity_words':obj_entity_words, "label":raw_df.label })
test_df

,sub_entity_words,obj_entity_words,label
0,비틀즈,조지 해리슨,no_relation
1,민주평화당,대안신당,no_relation
2,광주FC,한국프로축구연맹,org:member_of
3,아성다이소,박정부,org:top_members/employees
4,요미우리 자이언츠,1967,no_relation
...,...,...,...
32465,유기준,부산 서구·동구,per:employee_of
32466,최시형,손병희,per:colleagues
32467,완도군,신우철,org:top_members/employees
32468,JTBC,중앙홀딩스,no_relation


In [16]:
import re

In [51]:
p = re.compile("[가-힣]+")
tmp = []
for idx, s_e in enumerate(sub_entity_words):
    processed = re.sub("[가-힣]+|[0-9]+|[a-z]+|[A-Z]+|[.]+","", s_e)
    if processed.strip() != '':
        tmp.append((idx,s_e))
tmp
# 결과를 보면 () 괄호나 - 이런 기호들도 있다

[(114, 'CJ E&M'),
 (123, '툰-운트 스포트버라인 바이어 04 레버쿠젠'),
 (651, 'S★스파이시'),
 (780, 'SS-FHA'),
 (889, 'KT&G'),
 (1346, '나진 e-엠파이어'),
 (1414, 'MŠK 질리나'),
 (1465, 'GitHub, Inc.'),
 (1508, 'H&M'),
 (1537, 'KT&G'),
 (2010, '부산-김해경전철'),
 (2167, '메르세데스-벤츠'),
 (2418, '메르세데스-벤츠'),
 (3338, '샤르자(Sharjah) 대학'),
 (3999, '르네-로베르 카블리에 드 라 살'),
 (4116, 'FC 로잔-스포르트'),
 (4122, 'f(x)'),
 (4349, 'GD&TOP'),
 (4813, '칸쟈니∞'),
 (4957, '아래쪽 센타우루스자리-남십자자리 서브그룹'),
 (5011, 'KT&G'),
 (6082, 'G-Unit'),
 (6324, 'f(x)'),
 (6740, 'Oh!'),
 (7011, 'B*Witched'),
 (7649, '도네츠크-크리보이로크 소비에트 공화국'),
 (7852, 'EXO-M'),
 (7911, 'EXO-M'),
 (7964, '프리울리-베네치아'),
 (9176, 'f(x)'),
 (9243, 'KT&G'),
 (9265, '현대로템(주)'),
 (9540, 'f(x)'),
 (9590, '민주변화운동 - 창기라이'),
 (9766, 'US-방글라 항공'),
 (10330, '도네츠크-크리보이로크 소비에트 공화국'),
 (10636, 'K-water'),
 (10773, '브라이턴 & 호브 앨비언'),
 (11137, 'SM C&C'),
 (11267, '(주)마인버스'),
 (11466, 'k-os'),
 (11530, '메르세데스-벤츠'),
 (11739, 'pH-1'),
 (12175, '르네-로베르 카블리에 드 라 살'),
 (12416, 'GD&TOP'),
 (12477, '동원F&B'),
 (12522, 'D

In [52]:
tmp = np.array(tmp)
uniq_sub = raw_df.iloc[tmp[:,0]]
uniq_sub.to_csv(os.path.join(SAVEDIR,'unique_sub_entity.csv'))
uniq_sub

,id,sentence,subject_entity,object_entity,label,source
114,114,"2018년 7월 1일 CJ E&M을 합병하여 CJ ENM으로 사명을 변경하으며, 현...","{'word': 'CJ E&M', 'start_idx': 12, 'end_idx':...","{'word': 'CJ오쇼핑', 'start_idx': 48, 'end_idx': ...",no_relation,wikipedia
123,123,"1904년 7월 1일, ""툰-운트 스포트버라인 바이어 04 레버쿠젠"" (Turn- ...","{'word': '툰-운트 스포트버라인 바이어 04 레버쿠젠', 'start_idx...","{'word': '1904년 7월 1일', 'start_idx': 0, 'end_i...",org:founded,wikipedia
651,651,S★스파이시(S★Spicy)는 스타더스트 프로모션에 소속된 일본의 여성 아이돌 그룹이다.,"{'word': 'S★스파이시', 'start_idx': 0, 'end_idx': ...","{'word': '스타더스트 프로모션', 'start_idx': 17, 'end_i...",org:member_of,wikipedia
780,780,"그 결과 SS-HA는 대폭 감축당했고 친위대 지도본부(SS-FHA), 국가보안본부(...","{'word': 'SS-FHA', 'start_idx': 30, 'end_idx':...","{'word': '친위대 지도본부', 'start_idx': 21, 'end_idx...",org:alternate_names,wikipedia
889,889,KT&G(사장 백복인)가 궐련형 전자담배 ‘릴 하이브리드(lil HYBRID) 1....,"{'word': 'KT&G', 'start_idx': 0, 'end_idx': 3,...","{'word': '백복인', 'start_idx': 8, 'end_idx': 10,...",org:top_members/employees,wikitree
...,...,...,...,...,...,...
29836,29836,"1882년 오스트리아-헝가리 제국에 있던 사설 철도가 국영화되기 시작하였는데, 그 ...","{'word': '오스트리아-헝가리 제국', 'start_idx': 6, 'end_...","{'word': '오스트리아 제국', 'start_idx': 49, 'end_idx...",no_relation,wikipedia
30017,30017,마창시내버스협의회 소속 가운데 가장 많은 버스대수를 보유하고 있었던 마산의 (유)시...,"{'word': '(유)시민버스', 'start_idx': 42, 'end_idx'...","{'word': '시내버스협의회', 'start_idx': 2, 'end_idx':...",org:member_of,wikipedia
30787,30787,"단독 호스트 한혜진 씨를 비롯해 김재환, 마마무(MAMAMOO), 우주소녀(WJSN...","{'word': 'IZ*ONE', 'start_idx': 139, 'end_idx'...","{'word': 'K-POP', 'start_idx': 167, 'end_idx':...",no_relation,wikitree
31128,31128,"토머스 제퍼슨과 그가 이끄는 민주-공화당이 영국과의 조약을 반대하였지만, 워싱턴은 ...","{'word': '민주-공화당', 'start_idx': 16, 'end_idx':...","{'word': '토머스 제퍼슨', 'start_idx': 0, 'end_idx':...",org:top_members/employees,wikipedia


In [53]:
tmp = []
for idx, s_e in enumerate(obj_entity_words):
    processed = re.sub("[가-힣]+|[0-9]+|[a-z]+|[A-Z]+|[.]+","", s_e)
    if processed.strip() != '':
        tmp.append((idx,s_e))
tmp


[(45, '1800년 ~ 1862년'),
 (72, '49만 5,000명'),
 (107, '031-909-9000'),
 (136, 'KT 커스터머&미디어부문장'),
 (191, '少弐氏'),
 (261, '25,000'),
 (328, '2012-13'),
 (342, '37,000명'),
 (374, '玉置浩二'),
 (418, 'KBS 全州放送總局'),
 (420, 'DFB-포칼'),
 (440, '6,000명'),
 (444, '군인, 탐험가'),
 (457, '1915년 9월 9일 ~ 1958년 11월 19일'),
 (494, '씨제이이에치(주)'),
 (540, '後北条氏'),
 (627, '제1, 3야전군'),
 (630, "시동'"),
 (714, '18,000명'),
 (723, '스페이스워!'),
 (724, '2008-09'),
 (750, '수원 삼성 블루윙즈 U-15'),
 (893, '2006-07'),
 (926, '1426년 5월 5일/음력 3월 28일'),
 (964, '2002-03'),
 (977, '16·17·18·20'),
 (1047, 'V-리그'),
 (1052, '22,528.74 위안'),
 (1053, '031-909-9000'),
 (1055, 'VMRO-DPMNE'),
 (1151, '20,000명'),
 (1291, 'AT&T 스타디움'),
 (1293, 'Hermann Müller'),
 (1299, 'STS-107'),
 (1310, '前 새누리당 비례대표'),
 (1466, '93,000명'),
 (1476, '2019-2020'),
 (1481, 'USANA Health Sciences, Inc.'),
 (1485, '서울動物園'),
 (1506, 'SM C&C'),
 (1527, '서울특별시 종로구 종로 26 (서린동)'),
 (1574, 'A-JAX'),
 (1579, '1925년 4월 14일 ~'),
 (1720, '2017-18'),
 (1729, '성삼빙, 성삼고, 성삼성'),
 (1756

In [54]:
tmp = np.array(tmp)
uniq_obj = raw_df.iloc[tmp[:,0]]
uniq_obj.to_csv(os.path.join(SAVEDIR,'unique_obj_entity.csv'))
uniq_obj

,id,sentence,subject_entity,object_entity,label,source
45,45,영국의 제임스 클라크 로스(1800년 ~ 1862년) 경은 1839년 10월 ‘에레...,"{'word': '제임스 클라크 로스', 'start_idx': 4, 'end_id...","{'word': '1800년 ~ 1862년', 'start_idx': 15, 'en...",per:date_of_birth,wikipedia
72,72,냉전 동안 독일 연방군은 NATO의 서유럽 방위의 주력군이 되었으며 전군을 통틀어 ...,"{'word': '독일 연방군', 'start_idx': 6, 'end_idx': ...","{'word': '49만 5,000명', 'start_idx': 46, 'end_i...",org:number_of_employees/members,wikipedia
107,107,고양시청 종합상황실에서는 연휴기간동안 09:00 ~ 18:00까지 민원콜센터(031...,"{'word': '고양시', 'start_idx': 0, 'end_idx': 2, ...","{'word': '031-909-9000', 'start_idx': 43, 'end...",no_relation,wikitree
136,136,KT 이사회가 전원합의로 구현모 KT 커스터머&미디어부문장(사장)을 차기 회장 후보...,"{'word': '구현모', 'start_idx': 14, 'end_idx': 16...","{'word': 'KT 커스터머&미디어부문장', 'start_idx': 18, 'e...",per:title,wikitree
191,191,쇼니 씨(少弐氏)의 8대 당주로 쇼니 요리히사(少弐頼尚)의 둘째 아들이다.,"{'word': '쇼니 씨', 'start_idx': 0, 'end_idx': 3,...","{'word': '少弐氏', 'start_idx': 5, 'end_idx': 7, ...",per:alternate_names,wikipedia
...,...,...,...,...,...,...
32218,32218,뱅가드 대학교의 운동부는 '라이언스(Lions)'로 미국 대학 대항 체육 협회에 소...,"{'word': '뱅가드 대학교', 'start_idx': 0, 'end_idx':...","{'word': '라이언스(Lions)', 'start_idx': 15, 'end_...",org:members,wikipedia
32228,32228,"정청래(鄭淸來, 1965년 4월 18일 -)는 대한민국의 운동권 출신 정치인으로, ...","{'word': '정청래', 'start_idx': 0, 'end_idx': 2, ...","{'word': '제17·19·21대', 'start_idx': 46, 'end_i...",no_relation,wikipedia
32301,32301,"잭 앵글린(Jack Anglin, 1916년 5월 13일 ~ 1963년 3월 8일)...","{'word': '잭 앵글린', 'start_idx': 0, 'end_idx': 4...","{'word': '가수, 작사가, 기타 연주자', 'start_idx': 59, '...",per:title,wikipedia
32411,32411,"가톨릭평화방송(가톨릭平和放送, Catholic Peace Broadcasting C...","{'word': '가톨릭평화방송', 'start_idx': 0, 'end_idx':...","{'word': '가톨릭平和放送', 'start_idx': 8, 'end_idx':...",org:alternate_names,wikipedia
